In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import t

D = 0.0060  # m, tube diameter
rho = 1.2  # kg/m^3, air density
mu = 1.83e-5  # Pa s, air viscosity
rhow = 1000  # kg/m^3, water density
g = 9.8  # m/s^2, gravitational acceleration

#######################INPUT VALUES HERE###############################
v = np.array([9.82, ])  # INPUT VELOCITY in m/s
dh = np.array([0.0198, ])  # INPUT HEIGHT DIFFERENCE in m
error_in_v =   # INPUT ERROR IN VELOCITY in %
error_in_dh =   # INPUT ERROR IN HEIGHT DIFFERENCE in %
#######################################################################

Re = v * D * rho / mu
P = rhow * g * dh
fexpt = P / (2 * rho * v**2 * (0.50 / D))

def calculate_error_bounds(v, dh):
    v_lower = v * (1-error_in_v/100)
    v_upper = v * (1+error_in_v/100)
    dh_lower = dh * (1-error_in_dh/100)
    dh_upper = dh * (1+error_in_dh/100)

    Re_lower = v_lower * D * rho / mu
    Re_upper = v_upper * D * rho / mu

    P_lower = rhow * g * dh_lower
    P_upper = rhow * g * dh_upper

    f_lower = P_lower / (2 * rho * v_upper**2 * (0.50 / D))
    f_upper = P_upper / (2 * rho * v_lower**2 * (0.50 / D))

    return Re_lower, Re_upper, f_lower, f_upper

Re_lower, Re_upper, f_lower, f_upper = calculate_error_bounds(v, dh)

f_theory = lambda Re: 0.0791 * Re**(-0.25)

def f_model(Re, C1, C2):
    return C1 * Re**C2

initial_guess = [0.08, -0.25]
params, covariance = curve_fit(f_model, Re, fexpt, p0=initial_guess)
C1, C2 = params
errors = np.sqrt(np.diag(covariance))
C1_error, C2_error = errors

# Monte Carlo simulation
def monte_carlo_analysis(v, dh, num_simulations=1000):
    np.random.seed(42)

    synthetic_C1 = []
    synthetic_C2 = []

    for _ in range(num_simulations):
        v_simulated = v + np.random.normal(0, error_in_v/100 * v, size=len(v))
        dh_simulated = dh + np.random.normal(0, error_in_dh/100 * dh, size=len(dh))

        Re_simulated = v_simulated * D * rho / mu
        P_simulated = rhow * g * dh_simulated
        f_simulated = P_simulated / (2 * rho * v_simulated**2 * (0.50 / D))

        try:
            params_simulated, _ = curve_fit(f_model, Re_simulated, f_simulated, p0=initial_guess)
            synthetic_C1.append(params_simulated[0])
            synthetic_C2.append(params_simulated[1])
        except:
            continue

    return np.array(synthetic_C1), np.array(synthetic_C2)

synthetic_C1, synthetic_C2 = monte_carlo_analysis(v, dh)

C1_lower = np.percentile(synthetic_C1, 2.5)
C1_upper = np.percentile(synthetic_C1, 97.5)
C2_lower = np.percentile(synthetic_C2, 2.5)
C2_upper = np.percentile(synthetic_C2, 97.5)

params_summary = pd.DataFrame({
    "Parameter": ["C1", "C2"],
    "Value": [round(C1, 4), round(C2, 3)],
    "95% CI Lower": [round(C1_lower, 3), round(C2_lower, 3)],
    "95% CI Upper": [round(C1_upper, 3), round(C2_upper, 3)]
})

plt.figure(figsize=(5, 5))
plt.errorbar(Re, fexpt, xerr=[Re - Re_lower, Re_upper - Re], yerr=[fexpt - f_lower, f_upper - fexpt], fmt='o', markersize=8, color='tab:blue', alpha=1, capsize=3)
plt.plot(Re, f_model(Re, C1, C2), color='tab:blue', linestyle='-', label='Best-fit model $f = C_1 \mathrm{Re}^{C_2}$')
plt.plot(Re, f_theory(Re), color='tab:orange', linestyle='--', label='$f_\mathrm{theory} = 0.0791 \mathrm{Re}^{-0.25}$')

plt.title('Analysis of friction factor for turbulent air flow in a plastic tube', pad=20)
plt.xlabel('Reynolds number (Re)')
plt.ylabel('Friction factor ($f$)')
plt.ylim(0.0, 0.02)
plt.xlim(0, 10000)
plt.yticks(np.arange(0.0, 0.021, 0.005))
plt.minorticks_on()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.grid(True, which='minor', linestyle=':', linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.savefig("friction_factor_plot.png", dpi=300)
plt.show()

plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
plt.hist(synthetic_C1, bins=30, color='tab:olive', alpha=0.7, edgecolor='black')
plt.axvline(C1, color='red', linestyle='--', label=f'Best-fit $C_1 = {C1:.4f}$')
plt.title('Distribution of $C_1$')
plt.xlabel('$C_1$')
plt.ylabel('Frequency')
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(synthetic_C2, bins=30, color='tab:cyan', alpha=0.7, edgecolor='black')
plt.axvline(C2, color='red', linestyle='--', label=f'Best-fit $C_2 = {C2:.3f}$')
plt.title('Distribution of $C_2$')
plt.xlabel('$C_2$')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
plt.show()

params_summary

In [ ]:
# from google.colab import files
# files.download("friction_factor_vs_reynolds.png")
# files.download("distribution_C1.png")
# files.download("distribution_C2.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>